In [ ]:
#Sentences have keywords in them 

In [ ]:
import os, re
import pandas as pd
import numpy as np
import time

# pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_core_sci_sm-0.4.0.tar.gz
# python -m spacy download en_core_web_sm
import spacy, en_core_sci_sm, en_core_web_sm
#from negspacy.negation import Negex


from collections import Counter
from math import sqrt 

In [ ]:
dc_sums=pd.read_csv('data/MIMIC_sentence_discharge_summaries.csv', names=['text'], header=None)
gen=pd.read_csv('data/MIMIC_sentence_general_notes.csv', names=['text'], header=None)
consult=pd.read_csv('data/MIMIC_sentence_consult_notes.csv', names=['text'], header=None)
physician=pd.read_csv('data/MIMIC_sentence_physician_notes.csv', names=['text'], header=None)
nursing=pd.read_csv('data/MIMIC_sentence_nursing_notes.csv', names=['text'], header=None)
resp=pd.read_csv('data/MIMIC_sentence_resp_notes.csv', names=['text'], header=None)
rehab=pd.read_csv('data/MIMIC_sentence_rehab_notes.csv', names=['text'], header=None)
nutrition=pd.read_csv('data/MIMIC_sentence_nutrition_notes.csv', names=['text'], header=None)

In [ ]:
all_keywords=[]
neg_keywords=pd.read_csv('data-raw/negative_keywords.csv').iloc[::,0]
pos_keywords=pd.read_csv('data-raw/positive_keywords.csv').iloc[::,0]

all_keywords.append(neg_keywords.apply(lambda x: x.strip()).tolist())
all_keywords.append(pos_keywords.apply(lambda x: x.strip()).tolist())

keywords=[i.lower() for i in all_keywords for i in i]
keywords.sort(key=lambda x: len(x.split()), reverse=True)

In [ ]:
"""
# This file is from the google drive.
# TODO: switch to positive/negative keyword files.
kw_df = pd.read_excel('data-raw/Keywords for lexicon-based sentiment classifier.xlsx',
                      sheet_name = 'Word Delineations')
kw_df.info()

# Sadly there is a space after "negative" and "positive" :/
kw_df.columns

#neg_keywords=pd.read_csv('data-raw/negative_keywords.csv').iloc[::,0]
neg_keywords = kw_df['Negative '].dropna().str.strip().str.lower().tolist()

print(neg_keywords[:5])

#pos_keywords=pd.read_csv('data-raw/positive_keywords.csv').iloc[::,0]
pos_keywords = kw_df['Positive '].dropna().str.strip().str.lower().tolist()
print(pos_keywords[:5])

kw = {'pos': pos_keywords,
            'neg': neg_keywords,
            'all': neg_keywords + pos_keywords}

# We need to have the longer phrases first to ensure that nested keywords
# are not detected multiple times. Sort by descending word count.
kw['all'].sort(key = lambda x: len(x.split()), reverse = True)
kw['all'][:5]
keywords = kw['all']
"""

In [ ]:
keywords

In [ ]:
#Example sentence 
text_samp='her respiratory distress was not improving, and she did not want elective intubation'
print(text_samp)

tokenized=text_samp.split()
tokenized

In [ ]:
found_words=[]
print(text_samp)
print('\nKeywords Found')
for i in keywords: 
    if re.search(r'\b{}\b'.format(i), text_samp):
        print(i)
        if i not in ' '.join(found_words):
            found_words.append(i)

print('\nFinal list of keywords:', found_words)


In [ ]:
def find_keywords(text, keywords):
    found_keywords=[]
    text=str(text).lower()
    for i in keywords: 
        if re.search(r'\b{}\b'.format(i), text): 
            if i not in ' '.join(found_keywords):
                found_keywords.append(i)
    return [text, found_keywords]

In [ ]:
#Test with example sentence
find_keywords(text_samp, keywords)

In [ ]:
# Test with sentence that contains both "not improving" and "improving".
test_str = "her respiratory distress was not improving initially, but with 6L nc it is now improving."
# Notice how the shorter keyword "improving' is not detected due to the algorithm.
find_keywords(test_str, keywords)

In [ ]:
from keywords import find_keywords2

In [ ]:
# Try a more complex algorithm that identifies nested keywords in the text.
# We can also use this to save the number of times each keyword appears, but taking
# the length of the dictionary value for that keyword (i.e. the number of locations extracted).
found_keywords = find_keywords2(test_str, keywords, verbose = True)
print("All keywords found: ", found_keywords)
list(found_keywords.keys())

In [ ]:
words = [word_i for word_i, locations in found_keywords.items()]
words

In [ ]:
%%time
# Takes 3 minutes.

final_array=[0,0]
for text in gen.text:
    #temp_array=find_keywords(text, keywords)
    initial_result = find_keywords2(text, keywords)
    temp_array = [text, [word_i for word_i, locations in initial_result.items()]]
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
gen_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)
print('total gen sentences', gen.shape[0])
print('total gen sentences with keywords', gen_final.shape[0])
gen_final.pop('keywords_found')
print('')
# with found_keywords: 
# total sentences: 106,127
# with keywords: 11,029
# with found_keywords2:
# total sentences:  106,127
# with keywords:  11,029

In [ ]:
temp_array

In [ ]:
final_array=[0,0]
for text in consult.text:
    #temp_array=find_keywords(text, keywords)
    initial_result = find_keywords2(text, keywords)
    temp_array = [text, [word_i for word_i, locations in initial_result.items()]]
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
cons_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1) 
print('total consult sentences', consult.shape[0])
print('total consult sentences with keywords', cons_final.shape[0])
cons_final.pop('keywords_found')
print('')

In [ ]:
%%time

final_array=[0,0]
# CK: why is this being restricted?
#for text in dc_sums.iloc[0:60000].text:
for text in dc_sums.text:
#    temp_array=find_keywords(text, keywords)
    initial_result = find_keywords2(text, keywords)
    temp_array = [text, [word_i for word_i, locations in initial_result.items()]]
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
dc_sums_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)    
print('total dc sum sentences', dc_sums.shape[0])
print('total dc sum sentences with keywords', dc_sums_final.shape[0])
dc_sums_final.pop('keywords_found')
print('')
# Previous results:
# 5,764,750; 4,654

In [ ]:
final_array=[0,0]
# CK: why is this being restricted?
for text in physician.iloc[0:60000].text:
#for text in physician.text:
#    temp_array=find_keywords(text, keywords)
    initial_result = find_keywords2(text, keywords)
    temp_array = [text, [word_i for word_i, locations in initial_result.items()]]
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
physician_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)    
print('total physician sentences', physician.shape[0])
print('total physician sentences with keywords', physician_final.shape[0])
physician_final.pop('keywords_found')
print('')

In [ ]:
final_array=[0,0]
for text in nursing.iloc[0:60000].text:
    temp_array=find_keywords(text, keywords)
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
nursing_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)    
print('total nursing sentences', nursing.shape[0])
print('total nursing sentences with keywords', nursing_final.shape[0])
nursing_final.pop('keywords_found')
print('')

In [ ]:
final_array=[0,0]
for text in resp.text:
    temp_array=find_keywords(text, keywords)
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
resp_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)    
print('total resp sentences', resp.shape[0])
print('total resp sentences with keywords', resp_final.shape[0])
resp_final.pop('keywords_found')
print('')

In [ ]:
final_array=[0,0]
for text in rehab.text:
    temp_array=find_keywords(text, keywords)
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
rehab_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)    
print('total rehab sentences', rehab.shape[0])
print('total rehab sentences with keywords', rehab_final.shape[0])
rehab_final.pop('keywords_found')
print('')

In [ ]:
final_array=[0,0]
for text in nutrition.text:
    temp_array=find_keywords(text, keywords)
    if len(temp_array[1])>0: 
        final_array=np.vstack((final_array, temp_array)) 
nutrition_final=pd.DataFrame(np.delete(final_array, 0, 0), columns=['text', 'keywords_found']).sample(frac=1)    
print('total nutrition sentences', nutrition.shape[0])
print('total nutrition sentences with keywords', nutrition_final.shape[0])
nutrition_final.pop('keywords_found')
print('')

In [ ]:
gen_final

In [ ]:
dc_sum_sample=dc_sums_final.sample(n=500).reset_index(drop=True)
physician_sample=physician_final.sample(n=500).reset_index(drop=True)
general_sample=gen_final.sample(n=500).reset_index(drop=True)
consult_sample=cons_final.sample(n=406).reset_index(drop=True)
nursing_sample=nursing_final.sample(n=500).reset_index(drop=True)
resp_sample=resp_final.sample(n=500).reset_index(drop=True)
rehab_sample=rehab_final.sample(n=500).reset_index(drop=True)
nutrition_sample=nutrition_final.sample(n=500).reset_index(drop=True)

In [ ]:
dc_sum_sample.to_csv('data/MIMIC_sentence_discharge_summaries.csv', index=False)
physician_sample.to_csv('data/MIMIC_sentence_physician_notes.csv', index=False)
general_sample.to_csv('data/MIMIC_sentence_general_notes.csv',index=False)
consult_sample.to_csv('data/MIMIC_sentence_consult_notes.csv',index=False)
nursing_sample.to_csv('data/MIMIC_sentence_nursing_notes.csv',index=False)
resp_sample.to_csv('data/MIMIC_sentence_resp_notes.csv',index=False)
rehab_sample.to_csv('data/MIMIC_sentence_rehab_notes.csv',index=False)
nutrition_sample.to_csv('data/MIMIC_sentence_nutrition_notes.csv',index=False)

In [ ]:
gen_final.to_csv('data/keywords_general_notes.csv')
cons_final.to_csv('data/keywords_consult_notes.csv')
resp_final.to_csv('data/keywords_respiratory_notes.csv')
rehab_final.to_csv('data/keywords_rehab_notes.csv')
nutrition_final.to_csv('data/keywords_nutrition_notes.csv')
dc_sums_final.to_csv('data/Desktop/keywords_dc_summaries.csv')
nursing_final.to_csv('data/Desktop/keywords_nursing_notes.csv')
physician_final.to_csv('data/Desktop/keywords_physician_notes.csv')